In [2]:
import serial

# Open the serial port
ser = serial.Serial('COM3', 9600)  # Adjust baud rate as needed

try:
    while True:
        # Read a line from the serial port
        data = ser.readline()
        
        # Decode the bytes to string using UTF-8 encoding
        decoded_data = data.decode('utf-8', errors='replace')
        
        # Print the received data
        print(decoded_data, end='')
        
except KeyboardInterrupt:
    # Close the serial port when KeyboardInterrupt (Ctrl+C) is detected
    ser.close()
    print("Serial port closed")


���Accelerometer Data - X: 254.76 g | Y: 0.59 g | Z: 0.17 g
GPS - No Valid Data
GPS - No Valid Data
GPS - No Valid Data
Accelerometer Data - X: 254.76 g | Y: 0.58 g | Z: 0.18 g
GPS - Latitude: 33.641860 | Longitude: 72.992616 | Altitude: 575.80
Accelerometer Data - X: 254.77 g | Y: 0.58 g | Z: 0.17 g
GPS - Latitude: 33.641848 | Longitude: 72.992626 | Altitude: 576.00
Accelerometer Data - X: 254.76 g | Y: 0.59 g | Z: 0.17 g
GPS - Latitude: 33.641838 | Longitude: 72.992641 | Altitude: 576.10
Accelerometer Data - X: 254.76 g | Y: 0.58 g | Z: 0.17 g
GPS - Latitude: 33.641867 | Longitude: 72.992641 | Altitude: 575.70
Accelerometer Data - X: 254.76 g | Y: 0.58 g | Z: 0.17 g
GPS - Latitude: 33.641917 | Longitude: 72.992612 | Altitude: 575.00
Accelerometer Data - X: 254.76 g | Y: 0.58 g | Z: 0.17 g
GPS - Latitude: 33.641947 | Longitude: 72.992600 | Altitude: 574.40
Accelerometer Data - X: 254.77 g | Y: 0.58 g | Z: 0.17 g
GPS - Latitude: 33.641939 | Longitude: 72.992613 | Altitude: 574.40
Accel

In [4]:
import serial
import re
import psycopg2
from datetime import datetime

# PostgreSQL database settings
DB_NAME = 'fyp_database'
DB_USER = 'postgres'
DB_PASSWORD = 'HayatKhan'
DB_HOST = 'fyp-database.cdesuu80g5zo.eu-north-1.rds.amazonaws.com'  # usually 'localhost' if running locally
DB_PORT = '5432'  # usually '5432' for PostgreSQL

# Open the serial port
ser = serial.Serial('COM3', 9600)  # Adjust baud rate as needed

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

# Create a cursor object
cur = conn.cursor()

# Regular expressions to extract data
gps_pattern = re.compile(r"GPS - Latitude: (\d+\.\d+) \| Longitude: (\d+\.\d+) \| Altitude: (\d+\.\d+)")
accelerometer_pattern = re.compile(r"Accelerometer Data - X: (\d+\.\d+) g \| Y: (\d+\.\d+) g \| Z: (\d+\.\d+) g")

# Initialize variables for GPS data
latitude = 0.0
longitude = 0.0
altitude = 0.0

try:
    while True:
        # Read a line from the serial port
        data = ser.readline().decode('latin1').strip()

        # Extract GPS data
        gps_match = gps_pattern.match(data)
        if gps_match:
            latitude = float(gps_match.group(1))
            longitude = float(gps_match.group(2))
            altitude = float(gps_match.group(3))

        # Extract accelerometer data
        accelerometer_match = accelerometer_pattern.match(data)
        if accelerometer_match:
            x_axis = float(accelerometer_match.group(1))
            y_axis = float(accelerometer_match.group(2))
            z_axis = float(accelerometer_match.group(3))

            # Insert data into the database
            cur.execute(
                "INSERT INTO tank_1 (timestamp, x_axis, y_axis, z_axis, latitude, longitude, altitude) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (datetime.now(), x_axis, y_axis, z_axis, latitude, longitude, altitude)
            )
            conn.commit()

except KeyboardInterrupt:
    # Close the serial port when KeyboardInterrupt (Ctrl+C) is detected
    ser.close()
    print("Serial port closed")
    # Close database connection
    cur.close()
    conn.close()


Serial port closed
